In [16]:
#near duplicate detection and elimination
#next steps - obtain more data
#near entity recognition - spacy
#relabeling

# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [17]:
data_path = '../data/training_test_data/'
data = pd.read_csv(data_path + 'after_stock_data_cleaned.csv')
colnames = ['headline','source','label']   
data.columns = colnames

data2 = pd.read_csv(data_path + 'labeled_data.csv')


In [18]:
#dataset specific cleansing and concatenation

#data[data['source'].str.find(' via ') > -1]['source'].str[:data['source'].str.find(' via ')]

for i, row in data[data['source'].str.find(' via ') > -1].iterrows():
  data.at[i,'source'] = data.at[i,'source'][:data.at[i,'source'].find(' via ')].strip()

data.head()

,headline,source,label
0,"Apple, Amazon, YouTube: Safer, faster &amp; le...",USA TODAY,0
1,5G Stocks: Will Amazon Dip Its Toes In?,InvestorPlace,0
2,Fear of Amazon Creates a Bargain in FedEx Stock,InvestorPlace,0
3,Amazon Buyout Buzz Draws Options Bulls to Grub...,Schaeffer's Investment Research,0
4,US Stock Market Overview – Stocks Surge With O...,FX Empire,0


In [19]:
sources = []
headlines = []

for i, row in data2.iterrows():
    sources.append(data2.at[i, 'headlines'][len(data2.at[i, 'headlines'])-data2.at[i, 'headlines'][::-1].find('-'):].strip())
    headlines.append(data2.at[i, 'headlines'][:len(data2.at[i, 'headlines'])-data2.at[i, 'headlines'][::-1].find('-')-1].strip())                                
    
sources[:10] 
headlines[:10] 

clean_data2 = pd.DataFrame({'headline':headlines, 'source':sources, 'label':data2['relevancy']}, columns=colnames)

In [20]:
#concatenate our data to their data

df_union = pd.concat([data, clean_data2])

df_union.head()

,headline,source,label
0,"Apple, Amazon, YouTube: Safer, faster &amp; le...",USA TODAY,0
1,5G Stocks: Will Amazon Dip Its Toes In?,InvestorPlace,0
2,Fear of Amazon Creates a Bargain in FedEx Stock,InvestorPlace,0
3,Amazon Buyout Buzz Draws Options Bulls to Grub...,Schaeffer's Investment Research,0
4,US Stock Market Overview – Stocks Surge With O...,FX Empire,0


In [21]:
###placeholder cell, shingle and near duplicate elimination
 
 
def get_shingles(doc, size):
    shingles = set()
    if len(doc) > size:
        for i in range(0, len(doc)-size+1):
            shingles.add(doc[i:i+size])
    return shingles
 
def jaccard(set1, set2):
    x = len(set1.intersection(set2))
    y = len(set1.union(set2))
    return x / float(y)

def remove_near_dups(df, textcol, threshold):
    shingles = []

    for text in df[textcol]:
        shingles.append(get_shingles(text, 4))
 
    df['shingles'] = shingles
    
    return df
    
    rows = []
    dups = {}

    for i, row in df.iterrows():
        if i < len(df):
            for j in range(i+1,len(df)):
                if jaccard(df.at[i, 'shingles'], df.at[j, 'shingles']) > threshold:
                    dups.append(j)
                    
    print(dups)
                    
    for i, row in df.iterrows():
        if i not in dups:
            rows.append(row[:-1])
    
    return pd.DataFrame(rows, columns = df.columns)
            
        
        #data2.at[i, 'headlines']
    
new_df = remove_near_dups(df_union, 'headline', .4)
print(len(df_union))
print(len(new_df))
                                      

#print(jaccard(shingles1, shingles2), file1, file2)

#for i in range(i, len(data_union['headline']):
#for j in range(i,len(data_union['headline'])):
#print(jaccard(shingles1, shingles2), file1, file2)

4093
4093


In [22]:
df_union['label'].value_counts()

1    2840
0    1253
Name: label, dtype: int64

In [23]:
import re

df_union['headline'] = df_union['headline'].str.replace("[^a-zA-Z]", " ")

In [24]:
#word_count = sum(len(headline) for headline in df_union['headline'])

#print(word_count)

In [25]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df_union[['label','headline']], stratify = df_union['label'], test_size = 0.2, random_state = 12)
print(str(len(df_trn))+ ',' + str(len(df_val)))

3274,819


In [26]:
unlabeled_headlines = pd.read_csv(r'C:\Users\amber\Documents\VSE\DAEN 690\repo\DAEN-690\data\raw_data\Nowigence Raw Data.csv')

unlabeled_headlines.columns = ['headline']

text_df = pd.DataFrame(pd.concat([df_union['headline'], unlabeled_headlines['headline']]))
text_df['label'] = 0

txt_trn, txt_val = train_test_split(text_df[['label','headline']], stratify = text_df['label'], test_size = 0.2, random_state = 12)

In [27]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = txt_trn, valid_df = txt_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [28]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


In [54]:
learn.save_encoder('ft_enc')
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [56]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.626695,0.579021,0.700456,02:30
1,0.646332,0.581565,0.709567,02:22
2,0.640828,0.576475,0.685649,02:17
3,0.638155,0.583266,0.692483,02:18
4,0.632785,0.610416,0.671982,02:10


KeyboardInterrupt: 